In [42]:
# Príprava textu
# ==============

import nltk
import string

#načítanie knihy 
kniha1 = open('./Rivers of Babylon.txt', 'r', encoding="utf8")
kniha1 = kniha1.read()

#rozdelenie textu na slová
nltk.download('punkt')
from nltk.tokenize import word_tokenize
slova = word_tokenize(kniha1.lower())

#odstránenie neabecedných znakov
neabecedne_znaky = string.punctuation #!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~
zoznam_slov = []   #všetky slová
for slovo in slova:
        for char in slovo:
            if char in neabecedne_znaky:
                slovo = slovo.replace(char, "")
        if slovo not in neabecedne_znaky:
            zoznam_slov.append(slovo)
print("Počet slov:", len(zoznam_slov), "unikátnych", len(set(zoznam_slov)))
slovnik= set(zoznam_slov) #unikátne slová

# konverzia slov na indexy
indexy_slov = {slovo: i for i, slovo in enumerate(slovnik)}
zoznam_indexov_slov = [indexy_slov[slovo] for slovo in zoznam_slov]

# Príprava trénovacej množiny
davka=5
pocet_slov = len(zoznam_slov) #pocet slov knihy
trenovacia_mnozina = [([zoznam_indexov_slov[i], zoznam_indexov_slov[i+1], zoznam_indexov_slov[i+2], zoznam_indexov_slov[i+3], zoznam_indexov_slov[i+4]], zoznam_indexov_slov[i+5]) for i in range(pocet_slov-davka) ]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\llack\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Počet slov: 90374 unikátnych 19679


In [43]:
#Trieda definujúca model neurónovej siete
#========================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

embedding_dim = 5  #veľkosť embedding vektorov
# Embedding (vložená vrstva) je skrytá vrstva NS ktorá
# mapuje vstupné informácie z vysoko do nízko rozmerného priestoru

class Model_NS(nn.Module):
  def __init__(self, pocet_slov, embedding_dim, davka):
    super(Model_NS, self).__init__()
    self.embeddings = nn.Embedding(pocet_slov, embedding_dim)
    self.linear1 = nn.Linear(davka * embedding_dim,128)  # lineárne transformácie ako vrstvy NS
    self.linear2 = nn.Linear(128, 512)
    self.linear3 = nn.Linear(512, pocet_slov)

   # pri predpovedaní informácie prúdia „dopredu“
   # zo vstupu cez skryté vrstvy na výstup
   # výstupom sú logaritmy pravdepodobností
  def forward(self, inputs):  # pri predpovedaní
    embs = self.embeddings(inputs).view((1,-1))
    out = F.relu(self.linear1(embs)) #lineárna aktivačná funkcia
    out = F.relu(self.linear2(out))
    out = self.linear3(out)
    #konvertuje číselné hodnoty na pravdepodobnosti
    log_pravdepodobnosti = F.log_softmax(out, dim=1) 
    return log_pravdepodobnosti

# model a výpis parametrov modelu
model = Model_NS(pocet_slov, embedding_dim, davka)
print(model)

Model_NS(
  (embeddings): Embedding(90374, 5)
  (linear1): Linear(in_features=25, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=90374, bias=True)
)


In [29]:
# === trénovanie modelu CPU ===
device = torch.device("cpu")

#nParametre CPU
import psutil
print("CPU")
print ("fyzické jadrá: ",psutil.cpu_count(logical=False))
print ("logické jadrá ",psutil.cpu_count())

CPU
fyzické jadrá:  20
logické jadrá  28


In [44]:
# === trénovanie modelu GPU ===
from torch.cuda import is_available

# Pokiaľ možno na GPU
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

In [45]:
from datetime import timedelta

priebezne_chyby = []  # chyby po každých nnn krokoch trénovania 

def format_td(seconds, digits=3):
    isec, fsec = divmod(round(seconds*10**digits), 10**digits)
    return f'{timedelta(seconds=isec)}.{fsec:0{digits}.0f}'


def trenovanie(model, trenovacie_data, n_iteracie, word_to_index):
  model.to(device)
  n_time = time.time()
  for i in range(n_iteracie):
    model.train()
    krok = 0
    kroky_minuta = 0
    akumulovana_chyba = 0  # na výpočet priemernej chyby po každých nnn krokoch 
    
    for feature, target in trenovacie_data:
      feature_tensor = torch.tensor([feature], dtype=torch.long)
      feature_tensor = feature_tensor.to(device)
      target_tensor = torch.tensor([target], dtype=torch.long)
      target_tensor = target_tensor.to(device)
      model.zero_grad()  #vynulovanie gradientov
      log_probs = model(feature_tensor)
      loss = criterion(log_probs, target_tensor)
      loss.backward()    #Spätný prechod pri aktualizácii parametrov
      optimizer.step()
      akumulovana_chyba += loss.item()
      krok+=1
      kroky_minuta+=1
        
      sec = (time.time()-start_time)
      sec1 = (time.time()-n_time)
    
      if sec1 >= 60:  #každú minutu
        model.eval()
        priemerna_chyba = akumulovana_chyba/kroky_minuta
       
        print("čas {} ".format(format_td(sec, digits=2)),
               i+1,"z",n_iteracie, "iterácií", krok,"krok z",pocet_slov,
               "Chyba: {:.3f}".format(priemerna_chyba),              
              )      
        akumulovana_chyba=0
        kroky_minuta = 0
        n_time = time.time()
    model.train()
  return model


In [46]:
model = Model_NS(pocet_slov, embedding_dim, davka)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01)

n_iteracie = 1

start_time = time.time()

model = trenovanie(model, trenovacia_mnozina, n_iteracie, indexy_slov)
sec = (time.time()-start_time)
mins = sec/60

print("Trénovanie modelu trvalo {} minút".format(mins), 
      "--- {} ".format(format_td(sec, digits=2)))  

# Doplnenie modelu
model.average_loss = priebezne_chyby
model.optimizer_state = optimizer.state_dict()
model.word_to_index = indexy_slov,
model.index_to_word = {i: word for i, word in enumerate(slovnik)}


čas 0:01:00.13  1 z 1 iterácií 11906 krok z 90374 Chyba: 9.033
čas 0:02:00.13  1 z 1 iterácií 23730 krok z 90374 Chyba: 8.533
čas 0:03:00.13  1 z 1 iterácií 35524 krok z 90374 Chyba: 8.515
čas 0:04:00.14  1 z 1 iterácií 47413 krok z 90374 Chyba: 8.369
čas 0:05:00.14  1 z 1 iterácií 59335 krok z 90374 Chyba: 8.237
čas 0:06:00.14  1 z 1 iterácií 71262 krok z 90374 Chyba: 8.083
čas 0:07:00.15  1 z 1 iterácií 83181 krok z 90374 Chyba: 8.274
Trénovanie modelu trvalo 7.606516683101654 minút --- 0:07:36.39 


In [18]:
zaciatok_poviedky = input('Zadaj prvých {} slov poviedky:\n'.format(davka))

top_slova = 3 # Slovo sa vyberie náhodne z top_k počtu
dlzka_poviedky = 100
poviedka = []

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

# osetrenie vstupu, treba zadať 5 slov, ktoré model pozná
try:
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)
except KeyError as error:
    print('niektorému zo slov zatiaľ nerozumiem\n zadaj slová z kníh na ktorých som sa učil nie ', error)
    if poviedka[0] != "":
        poviedka = poviedka[len(zaciatok_poviedky):]
    zaciatok_poviedky = input('zadaj slová ktorým rozumiem\n')
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)
except KeyError and RuntimeError:
    if poviedka[0] != "":
        poviedka = poviedka[len(zaciatok_poviedky):]
    zaciatok_poviedky = input('Zadal si {} slov namiesto {}!\n\nZadaj presný počet slov\n'.format(len(zaciatok_poviedky.split(" ")), 5))
    predikcia = predpoved(model, zaciatok_poviedky, dlzka_poviedky, top_slova)

print('\n=== Poviedka ===')
print(" ".join(poviedka))


Zadaj prvých 5 slov poviedky:
obloha čoraz častejšie dostáva farbu

=== Poviedka ===
obloha čoraz častejšie dostáva farbu sa zdržuje rácz a o za pyšného po snežných pláňach stále dohora stále na sever psy ktoré z mesta po zdá si napokon pred hotelom no čo sa fiškál nasťahujú som dve ako všetkým sa dážď rozhodne už dávno po meste si reštauráciou sa už vyriešila vlasti na všetko deň svoju prídu sa fiškál unaví samozrejme stranu a na fiškála nikde hypotéku sa budú na mesto to on rácz vymyslel potrebovať nepozerá na jeseň ale no čo čo povedať rácz a nebola kohosi spokojne hotelom ambassador kravatu sa nik do si a pevne švéd a po celom meste si fiškáľa
